In [1]:
# 1.csv Jan 1, 2013- April 5, 2022; 
# 2.csv Jan 1, 2003 – December 31, 2012 
# 3.csv Jan 1, 1993 - December 31, 2002 

In [25]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [26]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [27]:
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
set_month_year = lambda d : str(d.year)+str(d.month)
set_year = lambda d : d.year
set_month = lambda d : d.month

In [28]:
ncd_cols = ['DATE', 'REPORT_TYPE', 
            'DailyAverageDryBulbTemperature', 
            'DailyAverageStationPressure',
            'DailyAverageWindSpeed', 
            'DailyCoolingDegreeDays', 'DailyDepartureFromNormalAverageTemperature', 
            'DailyHeatingDegreeDays',
            'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 
            'DailyPeakWindDirection', 'DailyPeakWindSpeed', 
            'DailyPrecipitation', 
            'DailySustainedWindDirection', 'DailySustainedWindSpeed',
            'DailyWeather']

In [5]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols = ncd_cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols = ncd_cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols = ncd_cols) 


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,32,33,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,33,34,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df1['DATE'] = df1['DATE'].apply(shortened_date)
df2['DATE'] = df2['DATE'].apply(shortened_date)
df3['DATE'] = df3['DATE'].apply(shortened_date)


In [7]:
# SOD sum of day, delete hourly records 
df1 = df1[df1['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df2 = df2[df2['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df3 = df3[df3['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 


In [8]:
nenana_climate = pd.concat([df1,df2,df3]).reset_index()
nenana_climate['year'] = nenana_climate['DATE'].apply(set_year)
nenana_climate['month'] = nenana_climate['DATE'].apply(set_month)


In [9]:
years_index = sorted([y for y in nenana_climate['year'].unique()])
months_index = sorted([m for m in nenana_climate['month'].unique()])

In [10]:
nenana_climate['DATE'].max()

Timestamp('2022-01-01 00:00:00')

In [81]:
t= dict()
for y in years_index:
    t[y] = {}
    for m in months_index:
        t[y][m] =[]
        for v in nenana_climate[ (nenana_climate['year']==y) & (nenana_climate['month']==m) ].DailyWeather.values:
            if type(v)== str: 
                t[y][m]+= [y for y in v.split() ]

In [82]:
weather_types = pd.DataFrame.from_dict(t)
weather_types

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[BR, BR, SN, SN, BR]","[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...",[],[],[],[],"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...",...,"[RA, BR, RA, RA, RA, RA, RA, SN, BR, RA, SN, H...","[RA, SN, RA, SN, SN, RA, RA, RA, SN, HZ, FZFG,...","[SN, BR, SN, SN, RA, RA, BR, RA, UP, RA, RA, S...","[BR, HZ, RA, RA, RA, SN, FZRA, SN, BR, HZ, RA,...","[SN, BR, SN, BR, SN, BR, UP, BR, SN, SN, BR, S...","[BR, BR, UP, FZRA, SN, BR, RA, FZRA, SN, FZFG,...","[RA, SN, FZFG, BR, UP, FG, BR, UP, HZ, HZ, BR,...","[SN, BR, UP, HZ, SN, HZ, HZ, SN, HZ, BR, HZ, B...","[HZ, SN, BR, UP, UP, BR, HZ, HZ, HZ, SN, SN, HZ]","[SN, UP, HZ]"
2,"[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[BR, SN, BR, BR, SN]","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...",[],[],[],[],"[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...",...,"[RA, RA, RA, SN, BR, RA, BR, RA, SN, HZ, RA, S...","[UP, SN, BR, SN, BR, HZ, FZRA, SN, FZFG, BR, U...","[SN, BR, RA, HZ, RA, SN, RA, RA, RA, RA, HZ, R...","[RA, UP, RA, UP, SN, HZ, FZFG, BR, HZ, RA, FG,...","[FZFG, BR, UP, HZ, FG, FZFG, BR, HZ, FG, HZ, U...","[RA, RA, SN, BR, RA, BR, RA, SN, BR, UP, HZ, S...","[SN, BR, SN, BR, SN, BR, UP, RA, SN, HZ, BR, S...","[HZ, SN, HZ, SN, SN, FZFG, BR, HZ, FG, SN, BR,...","[SN, SN, HZ, SN, SN, SN, FG, BR, FZFG, BR, HZ,...",[]
3,"[SN, SN, SN, SN, SN, RA]","[SN, SN, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...",[],[],[],[],"[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...",...,"[SN, BR, RA, HZ, RA, BR, RA, SN, FZFG, BR, FG,...","[UP, SN, BR, HZ, SN, BR, HZ, SN, UP, UP, RA, U...","[SN, BR, RA, SN, BR, SN, BR, UP, SN, BR, UP, H...","[RA, SN, SN, SN, BR, HZ, HZ, UP, SN, HZ, RA, R...","[RA, RA, SN, SN, BR, UP, HZ, SN, BR, HZ, SN, B...","[HZ, SN, BR, UP, SN, FZFG, BR, FG, SN, BR, UP,...","[SN, BR, SN, BR, SN, BR, HZ, SN, BR, RA, SN, B...","[SN, SN, HZ, SN, HZ, HZ, HZ, SN, HZ, SN, HZ, S...","[SN, SN, SN, SN, HZ, SN, BR, HZ, SN, BR, SN, H...",[]
4,"[SN, SN, RA]","[SN, SN, SN, RA]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]",[],[],[],[],"[SN, SN, SN, BR, RA, RA, RA]","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...",...,"[SN, BR, RA, SN, SN, BR, HZ, SN, BR, HZ, RA, S...","[RA, RA, RA, RA, SN, BR, HZ, RA, SN, BR, RA, R...","[SN, HZ, RA, SN, RA, SN, BR, HZ, UP, SN, SN, B...","[SN, BR, SN, BR, UP, RA, RA, RA, RA, SN, BR, H...","[RA, RA, HZ, RA, RA, SN, BR, RA, SN, BR, UP, H...","[SN, BR, SN, RA, RA, RA, RA, RA, RA, RA, RA, R...","[RA, RA, RA, RA, RA, RA, BR, HZ, RA, RA, SN, B...","[SN, BR, HZ, RA, SN, FG, BR, UP, RA, SN, UP, H...","[SN, UP, HZ, SN, BR, SN, FZFG, BR, FG, RA, SN,...",[]
5,"[RA, FG, RA, RA]","[RA, RA, RA, RA, RA, RA]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...",[],[],[],[],"[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, HZ, RA, BR, HZ, BR, BR]",...,"[SN, BR, HZ, RA, SN, RA, BR, SN, FZFG, BR, UP,...","[RA, RA, RA, RA, HZ, BR, RA, RA, RA, RA, RA, R...","[RA, BR, BR, RA, RA, RA]","[RA, RA, BR, RA, RA, RA, RA, RA, BR, RA, RA, R...","[RA, RA, RA, RA, RA, RA, RA, RA, RA, BR, RA, R...","[RA, SN, BR, UP, SN, BR, RA, RA, RA, BR, RA, S...","[RA, SN, BR, RA, RA, RA, RA, RA, RA, RA, RA, F...","[RA, RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA]",[]
6,"[RA, SN]",[RA],"[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...",[],[],[],[],"[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...",...,"[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, HZ, R.

In [83]:
ind, col, val = [], [], []
for k,v in weather_types.items():
    ind.append(k)
    for k2, v2 in v.items():
        col.append(k2)
        val.append(v2)

In [92]:
weather_types = pd.DataFrame.from_dict(weather_types);
weather_types

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[BR, BR, SN, SN, BR]","[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...",[],[],[],[],"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...",...,"[RA, BR, RA, RA, RA, RA, RA, SN, BR, RA, SN, H...","[RA, SN, RA, SN, SN, RA, RA, RA, SN, HZ, FZFG,...","[SN, BR, SN, SN, RA, RA, BR, RA, UP, RA, RA, S...","[BR, HZ, RA, RA, RA, SN, FZRA, SN, BR, HZ, RA,...","[SN, BR, SN, BR, SN, BR, UP, BR, SN, SN, BR, S...","[BR, BR, UP, FZRA, SN, BR, RA, FZRA, SN, FZFG,...","[RA, SN, FZFG, BR, UP, FG, BR, UP, HZ, HZ, BR,...","[SN, BR, UP, HZ, SN, HZ, HZ, SN, HZ, BR, HZ, B...","[HZ, SN, BR, UP, UP, BR, HZ, HZ, HZ, SN, SN, HZ]","[SN, UP, HZ]"
2,"[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[BR, SN, BR, BR, SN]","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...",[],[],[],[],"[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...",...,"[RA, RA, RA, SN, BR, RA, BR, RA, SN, HZ, RA, S...","[UP, SN, BR, SN, BR, HZ, FZRA, SN, FZFG, BR, U...","[SN, BR, RA, HZ, RA, SN, RA, RA, RA, RA, HZ, R...","[RA, UP, RA, UP, SN, HZ, FZFG, BR, HZ, RA, FG,...","[FZFG, BR, UP, HZ, FG, FZFG, BR, HZ, FG, HZ, U...","[RA, RA, SN, BR, RA, BR, RA, SN, BR, UP, HZ, S...","[SN, BR, SN, BR, SN, BR, UP, RA, SN, HZ, BR, S...","[HZ, SN, HZ, SN, SN, FZFG, BR, HZ, FG, SN, BR,...","[SN, SN, HZ, SN, SN, SN, FG, BR, FZFG, BR, HZ,...",[]
3,"[SN, SN, SN, SN, SN, RA]","[SN, SN, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...",[],[],[],[],"[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...",...,"[SN, BR, RA, HZ, RA, BR, RA, SN, FZFG, BR, FG,...","[UP, SN, BR, HZ, SN, BR, HZ, SN, UP, UP, RA, U...","[SN, BR, RA, SN, BR, SN, BR, UP, SN, BR, UP, H...","[RA, SN, SN, SN, BR, HZ, HZ, UP, SN, HZ, RA, R...","[RA, RA, SN, SN, BR, UP, HZ, SN, BR, HZ, SN, B...","[HZ, SN, BR, UP, SN, FZFG, BR, FG, SN, BR, UP,...","[SN, BR, SN, BR, SN, BR, HZ, SN, BR, RA, SN, B...","[SN, SN, HZ, SN, HZ, HZ, HZ, SN, HZ, SN, HZ, S...","[SN, SN, SN, SN, HZ, SN, BR, HZ, SN, BR, SN, H...",[]
4,"[SN, SN, RA]","[SN, SN, SN, RA]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]",[],[],[],[],"[SN, SN, SN, BR, RA, RA, RA]","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...",...,"[SN, BR, RA, SN, SN, BR, HZ, SN, BR, HZ, RA, S...","[RA, RA, RA, RA, SN, BR, HZ, RA, SN, BR, RA, R...","[SN, HZ, RA, SN, RA, SN, BR, HZ, UP, SN, SN, B...","[SN, BR, SN, BR, UP, RA, RA, RA, RA, SN, BR, H...","[RA, RA, HZ, RA, RA, SN, BR, RA, SN, BR, UP, H...","[SN, BR, SN, RA, RA, RA, RA, RA, RA, RA, RA, R...","[RA, RA, RA, RA, RA, RA, BR, HZ, RA, RA, SN, B...","[SN, BR, HZ, RA, SN, FG, BR, UP, RA, SN, UP, H...","[SN, UP, HZ, SN, BR, SN, FZFG, BR, FG, RA, SN,...",[]
5,"[RA, FG, RA, RA]","[RA, RA, RA, RA, RA, RA]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...",[],[],[],[],"[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, HZ, RA, BR, HZ, BR, BR]",...,"[SN, BR, HZ, RA, SN, RA, BR, SN, FZFG, BR, UP,...","[RA, RA, RA, RA, HZ, BR, RA, RA, RA, RA, RA, R...","[RA, BR, BR, RA, RA, RA]","[RA, RA, BR, RA, RA, RA, RA, RA, BR, RA, RA, R...","[RA, RA, RA, RA, RA, RA, RA, RA, RA, BR, RA, R...","[RA, SN, BR, UP, SN, BR, RA, RA, RA, BR, RA, S...","[RA, SN, BR, RA, RA, RA, RA, RA, RA, RA, RA, F...","[RA, RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA]",[]
6,"[RA, SN]",[RA],"[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...",[],[],[],[],"[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...",...,"[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, HZ, R.

In [93]:
weather_types.columns

Int64Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
            2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
            2018, 2019, 2020, 2021, 2022],
           dtype='int64')

In [94]:
weather_types.index

Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'month'], dtype='object')

In [95]:
events = pd.read_sql("""
    SELECT * 
    from nib;
    """, con=conn)
events

,Year,Decimal Day of Year,Date Time,month,TOD,TOY
0,1917,120.4792,04/30/1917 11:30:00 AM,04,0.4792,120.0
1,1918,131.3979,05/11/1918 09:33:00 AM,05,0.3979,131.0
2,1919,123.6063,05/03/1919 02:33:00 PM,05,0.6063,123.0
3,1920,132.4486,05/11/1920 10:46:00 AM,05,0.4486,132.0
4,1921,131.2792,05/11/1921 06:42:00 AM,05,0.2792,131.0
...,...,...,...,...,...,...
99,2016,114.6521,04/23/2016 03:39:00 PM,04,0.6521,114.0
100,2017,121.5000,05/01/2017 12:00:00 PM,05,0.5000,121.0
101,2018,121.5542,05/01/2018 01:18:00 PM,05,0.5542,121.0
102,2019,104.0146,04/14/2019 12:21:00 AM,04,0.0146,104.0


In [96]:
labels = list( events[events['Year']>=1996].month.values)
labels.append('?')
labels.append('?')
weather_types = weather_types.transpose()
weather_types['month'] = labels

In [97]:
weather_types

,1,2,3,4,5,6,7,8,9,10,11,12,month
1996,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN, RA]","[SN, SN, RA]","[RA, FG, RA, RA]","[RA, SN]","[RA, RA]",[],"[SN, RA, SN, SN]","[RA, SN, SN, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN, SN, SN]","[SN, SN, SN, SN, SN]",05
1997,"[BR, BR, SN, SN, BR]","[BR, SN, BR, BR, SN]","[SN, SN, SN]","[SN, SN, SN, RA]","[RA, RA, RA, RA, RA, RA]",[RA],"[RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, R...",[],[],[],[],04
1998,"[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, RA, RA, BR, RA, RA, BR, RA, RA, RA, BR, R...","[RA, RA, BR, RA, RA, RA, BR, RA, BR, RA, BR, R...","[RA, RA, RA, BR, FG, BR, RA, RA, BR, FG, BR, B...","[SN, BR, SN, BR, SN, BR, FG, BR, SN, SN, BR, S...","[BR, SN, SN, SN, HZ, SN, BR, SN, BR, SN, BR, S...","[SN, BR, SN, BR, SN, BR, SN, BR, SN, BR, SN, B...",04
1999,"[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...","[HZ, HZ, RA, HZ, RA, RA, RA, RA, BR, RA, BR, R...","[RA, RA, BR, RA, RA, BR, FG, BR, RA, RA, BR, R...","[RA, RA, BR, FG, BR, RA, BR, RA, RA, BR, RA, R...","[TS, FZRA, SN, BR, SN, BR, SN, BR, SN, BR, FG,...","[SN, BR, HZ, SN, BR, SN, BR, SN, BR, SN, BR, S...","[SN, SN, SN, BR, SN, SN, SN, SN, SN, SN, SN]",04
2000,[],[],[],[],[],[],[],[],[],[],[],[],05
2001,[],[],[],[],[],[],[],[],[],[],[],[],05
2002,[],[],[],[],[],[],[],[],[],[],[],[],05
2003,[],[],[],[],[],[],[],[],[],[],[],[],04
2004,"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, SN, SN, BR, RA, RA, RA]","[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[FG, BR, HZ, BR, HZ, RA, BR, HZ, RA, BR, HZ, R...","[RA, BR, HZ, BR, HZ, BR, HZ, BR, HZ, BR, HZ, F...","[RA, BR, HZ, RA, BR, BR, BR, HZ, BR, HZ, FG, B...","[SN, SN, SN, SN, BR, SN, BR, BR, SN, BR, SN, F...","[SN, BR, SN, BR, SN, BR, SN, BR, SN, BR, SN, B...","[SN, BR, SN, BR, SN, BR, SN, BR, HZ, SN, SN, S...",04
2005,"[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...","[RA, HZ, RA, BR, HZ, BR, BR]","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...","[BR, HZ, BR, HZ, BR, BR, HZ, HZ, BR, HZ, HZ, B...","[RA, RA, BR, BR, HZ, RA, BR, HZ, RA, BR, BR, H...","[RA, UP, RA, FZFG, BR, UP, FG, RA, RA, RA, RA,...","[BR, SN, HZ, RA, SN, UP, RA, UP, SN, UP, SN, S...","[SN, FZFG, BR, UP, HZ, FG, SN, FZFG, BR, FG, U...","[SN, BR, HZ, BR, HZ, SN, SN, BR, UP, UP, UP, S...",04


In [98]:
weather_types.to_csv('multinomial_features.csv', index=True)